In [2]:
from gridworld_env import gridWorld_env
from cursor1D_env import cursor1D_env

import numpy as np
from matplotlib import pyplot as plt

# Tabular Method Q-Learning

In [57]:
#Q saved is a list of Q matrices
#Each Q at index i is the Q estimate at episode number i

def runQLearning(env, learning_rate,discount_factor, num_of_episodes, num_steps_per_episode, Q0, l, explore_type='linear', explore_const = 0.2):
    #%% code starts here
    
    Q_saved = [Q0.copy()]
    Q = Q0.copy()
    
    for episode in range(num_of_episodes):
        #Start new episode
        state = env.reset()
        
        exploration = 0
        
        #Exploration rate:
        if explore_type == 'linear':
            exploration = 1 - episode/num_of_episodes
        elif explore_type == 'logarithmic':
            exploration = 1000.0/(1000.0 + episode)
        elif explore_type == 'constant':
                exploration = explore_const
        else:
            print("Invalid exploration type.")
            return
        
        for _ in range(num_steps_per_episode):
            
            #Perform an epsilon greedy action
            action = 0;
            r = np.random.rand()
            if r > exploration:
                action = np.argmax(Q0[state, :])
            else:
                action = env.action_space.sample()
            
            nextState, reward, done, _ = env.step(action, l)
            
            #Update Q
            Q[state, action] = Q[state, action] + learning_rate*(reward + discount_factor*np.max(Q[nextState, :]) - Q[state, action])
            
            state = nextState;
    
        #Store Q into Q_saved
        Q_saved.append(Q.copy())
        
    return Q_saved

In [3]:
#This tests the policy. Gives number of steps to reach goal and if goal is reached

def testPolicy(env, policy, max_num_steps = 30, render = False):
    s = env.reset()
    
    if render:
        env.render()
    
    num_steps = 0
    
    for _ in range(max_num_steps):
        
        a = int(policy[s])
        s, r, d, _ = env.step(a, l)
        
        if render:
            env.render()
        
        num_steps += 1
        
        if d == True:
            break
            
    return num_steps, d

# Gets baseline results for gridworld env

In [ ]:
#This is baseline performance of our solution

num_s = 16
num_a = 4

optimal_number_steps = 6

num_participants = 100
num_steps_per_participant = 100

#We are going to test where truePositiveRate = 1 - falsePositiveRate
rate = np.arange(0.50, 1.01, 0.05)
num_times_to_run = 1000

mean_num_participants_to_converge = []
std_num_participants_to_converge = []

mean_num_participants_to_solve = []
std_num_participants_to_solve = []

for r in rate:
    env = gridWorld_env(truePositiveRate = r, falsePositiveRate = 1-r)

    #We want to find statistics for number of participants required to get optimal_number_steps
    #And number of participants needed to simply solve the environment
    num_participants_to_converge_list = [];
    num_participants_to_solve_list = []

    for _ in range(num_times_to_run):
        #Run Q learning
        Q0 = np.zeros((num_s, num_a))
        Q_saved = runQLearning(env, 0.02,0.95, num_participants, num_steps_per_participant, Q0, 1,  explore_type = 'constant', explore_const = 1.0)

        steps_all = []
        done_all = []

        policy = np.zeros((num_s,))
        
        
        #Get the sucess rates
        for i in range(len(Q_saved)):
            Q = Q_saved[i]    

            #Get the policy from that Q
            for s in range(num_s):
                policy[s] = np.argmax(Q[s,:])
                

            steps, done = testPolicy(env, policy)

            steps_all.append(steps)
            done_all.append(done)
        
        
        #Check how many participants were needed to converge
        # There is a plus 1 here because the first Q in Q_saved is from Q0 (so 0 participants)
        #Also so when it breaks from the loop searching, it will mean we needed the next participant to converge.
        num_participants_to_converge = num_participants + 1
        for step in steps_all[::-1]:
            if step != optimal_number_steps:
                break
            num_participants_to_converge -= 1

        num_participants_to_solve = num_participants + 1
        for done in done_all[::-1]:
            if not done:
                break
            num_participants_to_solve -= 1
        
        num_participants_to_converge_list.append(num_participants_to_converge)
        num_participants_to_solve_list.append(num_participants_to_solve)


    num_participants_to_converge_list = np.array(num_participants_to_converge_list)
        
    mean_num_participants_to_converge.append(np.mean(num_participants_to_converge_list))
    std_num_participants_to_converge.append(np.std(num_participants_to_converge_list))
    
    num_participants_to_solve_list = np.array(num_participants_to_solve_list)
    
    mean_num_participants_to_solve.append(np.mean(num_participants_to_solve_list))
    std_num_participants_to_solve.append(np.std(num_participants_to_solve_list))

    
    print("r = " + str(r))
    print("mean to converge = " + str(mean_num_participants_to_converge[-1]))
    print("mean to solve    = " + str(mean_num_participants_to_solve[-1]))

In [ ]:
mean_num_participants_to_converge = np.array(mean_num_participants_to_converge)
std_num_participants_to_converge = np.array(std_num_participants_to_converge)

plt.figure(1)
plt.errorbar(rate, mean_num_participants_to_converge, std_num_participants_to_converge, fmt='-og', lw=1)

plt.title('Number of Participants Required to Find Optimal Solution')
plt.xlabel('Rate of error')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)

mean_num_participants_to_solve = np.array(mean_num_participants_to_solve)
std_num_participants_to_solve = np.array(std_num_participants_to_solve)
max_num_participants_to_solve = np.array(max_num_participants_to_solve)

plt.figure(2)
plt.errorbar(rate, mean_num_participants_to_solve, std_num_participants_to_solve, fmt='-og', lw=1)

plt.title('Number of Participants Required to Find A Solution')
plt.xlabel('Rate of error')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)

# Set hueristic reward for grid world

In [ ]:
# Subclass to overwrite the generalized reward:

class gridWorld_env1(gridWorld_env):
    
    def __init__(self, truePositiveRate = 0.7, falsePositiveRate = 0.3):
        super(gridWorld_env1,self).__init__(truePositiveRate, falsePositiveRate)
        
        
        # a = 0 is left, a = 1 is down, a = 2 is right, a = 3 is up
        #Below defines the hueristic reward. 1 is where we give reward, 0 is not
        R = [[0, 1, 1, 0], [0, 1, 1, 0], [0, 1, 1, 0], [1, 1, 1, 0],
             [0, 1, 1, 0], [0, 1, 1, 0], [0, 1, 1, 0], [1, 1, 1, 0],
             [0, 1, 1, 0], [0, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1],
             [0, 1, 1, 1], [0, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]
        
        self.R = np.array(R)
        
        self.k = self.falsePositiveRate*(self.R.size/np.count_nonzero(self.R))
        
    def getHeuristicReward(self, s,  a, last_s):
        if self.R[s, a]:
            return self.k
        else:
            return 0

# Hueristic Reward results for grid world

In [ ]:
#This is baseline performance of our solution

num_s = 16
num_a = 4

optimal_number_steps = 6
num_participants = 100
num_steps_per_participant = 100

num_trial_to_train_initial = 1000


#We are going to test where truePositiveRate = 1 - falsePositiveRate
rate = np.arange(0.50, 1.01, 0.05)
num_times_to_run = 1000

num_participants_to_solve_list_ours = []
num_participants_to_converge_list_ours = []

mean_num_participants_to_converge_ours = []
std_num_participants_to_converge_ours = []

mean_num_participants_to_solve_ours = []
std_num_participants_to_solve_ours = []

for r in rate:
    env = gridWorld_env1(truePositiveRate = r, falsePositiveRate = 1-r)
        
    
    #We want to find statistics for number of participants required to get optimal_number_steps
    #And number of participants needed to simply solve the environment
    num_participants_to_converge_list = [];
    num_participants_to_solve_list = []

    
    for _ in range(num_times_to_run):
        
        #Going to train initial Q0 here
        Q0 = np.zeros((num_s, num_a))
        for _ in range(num_trial_to_train_initial) :
            Q_init = runQLearning(env, 0.02 ,0.95, 1, num_steps_per_participant, Q0, 0,  explore_type = 'constant', explore_const = 1.0)
            Q0 = Q_init[-1]  

                    
        #Run Q learning
        Q_saved = runQLearning(env, 0.02,0.95, num_participants, num_steps_per_participant, Q0, 1,  explore_type = 'constant', explore_const = r)
        
        
        steps_all = []
        done_all = []

        policy = np.zeros((num_s,))
        
        
        #Get the sucess rates
        for i in range(len(Q_saved)):
            Q = Q_saved[i]    

            #Get the policy from that Q
            for s in range(num_s):
                policy[s] = np.argmax(Q[s,:])
                

            steps, done = testPolicy(env, policy)

            steps_all.append(steps)
            done_all.append(done)
        
        
        #Check how many participants were needed to converge
        # There is a plus 1 here because the first Q in Q_saved is from Q0 (so 0 participants)
        #Also so when it breaks from the loop searching, it will mean we needed the next participant to converge.
        num_participants_to_converge = num_participants + 1
        for step in steps_all[::-1]:
            if step != optimal_number_steps:
                break
            num_participants_to_converge -= 1

        num_participants_to_solve = num_participants + 1
        for done in done_all[::-1]:
            if not done:
                break
            num_participants_to_solve -= 1
        
        num_participants_to_converge_list.append(num_participants_to_converge)
        num_participants_to_solve_list.append(num_participants_to_solve)


    num_participants_to_converge_list_ours = np.array(num_participants_to_converge_list)
        
    mean_num_participants_to_converge_ours.append(np.mean(num_participants_to_converge_list))
    std_num_participants_to_converge_ours.append(np.std(num_participants_to_converge_list))
    
    num_participants_to_solve_list_ours = np.array(num_participants_to_solve_list)
    
    mean_num_participants_to_solve_ours.append(np.mean(num_participants_to_solve_list))
    std_num_participants_to_solve_ours.append(np.std(num_participants_to_solve_list))

    
    print("r = " + str(r))
    print("mean to converge = " + str(num_participants_to_converge_list_ours[-1]))
    print("mean to solve    = " + str(num_participants_to_solve_list_ours[-1]))

In [ ]:
# Plot baseline

mean_num_participants_to_converge = np.array(mean_num_participants_to_converge)
std_num_participants_to_converge = np.array(std_num_participants_to_converge)

plt.figure(1)
plt.errorbar(rate, mean_num_participants_to_converge, std_num_participants_to_converge, fmt='-og', lw=1)

plt.title('Number of Participants Required to Find Optimal Solution')
plt.xlabel('Rate of error')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)

mean_num_participants_to_solve = np.array(mean_num_participants_to_solve)
std_num_participants_to_solve = np.array(std_num_participants_to_solve)
max_num_participants_to_solve = np.array(max_num_participants_to_solve)

plt.figure(2)
plt.errorbar(rate, mean_num_participants_to_solve, std_num_participants_to_solve, fmt='-og', lw=1)

plt.title('Number of Participants Required to Find A Solution')
plt.xlabel('Rate of error')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)

#Plot our solution

mean_num_participants_to_converge = np.array(mean_num_participants_to_converge)
std_num_participants_to_converge = np.array(std_num_participants_to_converge)

plt.figure(1)
plt.errorbar(rate, mean_num_participants_to_converge, std_num_participants_to_converge, fmt='-ob', lw=1, label='Baseline')

mean_num_participants_to_solve = np.array(mean_num_participants_to_solve)
std_num_participants_to_solve = np.array(std_num_participants_to_solve)
max_num_participants_to_solve = np.array(max_num_participants_to_solve)

plt.figure(2)
plt.errorbar(rate, mean_num_participants_to_solve, std_num_participants_to_solve, fmt='-ob', lw=1, label='Baseline')


mean_num_participants_to_converge_ours = np.array(mean_num_participants_to_converge_ours)
std_num_participants_to_converge_ours = np.array(std_num_participants_to_converge_ours)

plt.figure(1)
plt.errorbar(rate, mean_num_participants_to_converge_ours, std_num_participants_to_converge_ours, fmt='-og', lw=1, label='Using Heuristic')
plt.legend()
plt.title('Number of Participants Required to Find Optimal Solution')
plt.xlabel('p1 = 1 - p0')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)

mean_num_participants_to_solve_ours = np.array(mean_num_participants_to_solve_ours)
std_num_participants_to_solve_ours = np.array(std_num_participants_to_solve_ours)

plt.figure(2)
plt.errorbar(rate, mean_num_participants_to_solve_ours, std_num_participants_to_solve_ours, fmt='-og', lw=1, label='Using Heuristic')
plt.legend()
plt.title('Number of Participants Required to Find Optimal Solution')
plt.xlabel('p1 = 1 - p0')
plt.ylabel('Number of Participants')
plt.ylim(-1, 101)
plt.xlim(0.49, 1.01)